In [ ]:
%%info

In [18]:
%%configure -f
{"executorCores": 6, "numExecutors":6}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
19,application_1527585229587_0033,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
19,application_1527585229587_0033,pyspark,idle,Link,Link,✔


In [1]:
# Load modules
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import PCA
from pyspark import SparkContext
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.sql.functions import *
from pyspark.sql import *
import numpy as np

from pyspark import SparkContext, SparkConf
conf = SparkConf().set("spark.executor.instances", "8").setAppName('appName').setMaster('spark://localhost:7077')
# Make sure one instance is running at once
sc = SparkContext.getOrCreate(conf=conf)
# Create spark session
spark = SparkSession.builder.getOrCreate()

In [ ]:
%%info

In [2]:
def createDF(path):
    # rename column '_c0' to 'label'
    f = spark.read.format('csv').options(header='false', inferSchema='true').load(path)
    data = f.withColumnRenamed("_c0","label")
    return data

def generatePCA(trainingset, PCA_k):
    transformer = VectorAssembler(inputCols=['_c%d' % i for i in range(1,785)],
                              outputCol="features")
    standardizer = StandardScaler(withMean=True, withStd=True,
                              inputCol='features',
                              outputCol='std_features')
    indexer = StringIndexer(inputCol="label", outputCol="label_idx")
    pca = PCA(k=PCA_k, inputCol="std_features", outputCol="pca")

    pipeline = Pipeline(stages=[transformer, standardizer, indexer, pca])
    model = pipeline.fit(trainingset)
    return model

In [3]:
#train = createDF('wasb://hdinsight2-2018-05-28t14-28-17-337z@ryanstorage10.blob.core.windows.net/Train-label-28x28.csv')
#test = createDF('wasb://hdinsight2-2018-05-28t14-28-17-337z@ryanstorage10.blob.core.windows.net/Test-label-28x28.csv')
train = createDF("/Users/dyin/Desktop/Cloudass2-master/Train-label-28x28.csv")
test = createDF("/Users/dyin/Desktop/Cloudass2-master/Test-label-28x28.csv")

In [9]:
# Create dataframe


# Generate model
print("Generating model")
model = generatePCA(train, 50)

# Train model
train_result = model.transform(train)
test_result = model.transform(test)

Generating model


In [ ]:
# Check trained results
print(train_result.take(2))
print(test_result.take(2))

In [ ]:
# transform full data back to rdd
train_result_rdd = train_result.select(["label", "pca"]).rdd
test_result_rdd = test_result.select(["label", "pca"]).rdd

#print(train_result_sample.take(1))
#x = train_result_sample.mapPartitions(calculateDistance(train_result_sample, test_result_sample, 2))

In [ ]:
print(train_result_rdd.take(1))

In [ ]:
# Run this section to use portional data rdd
# Take small sample from data: set variables to full data if not using parts of rdd for flexible computation
test_x_rdd = sc.parallelize([ row['pca'] for row in test_result_rdd.take(1000) ])
test_y_rdd = sc.parallelize([ row['label'] for row in test_result_rdd.take(1000) ])
train_x_rdd = sc.parallelize([ row['pca'] for row in train_result_rdd.take(1000) ])
train_y_rdd = sc.parallelize([ row['label'] for row in train_result_rdd.take(1000) ])

print(train_x.take(1))
print(test_x.take(1))
print(np.array(train_x.collect())[1,:])
print(np.array(test_x.collect())[1,:])
#print(np.array(test_y.collect()))

In [ ]:
# Run this section to use full data rdd

test_x_rdd = sc.parallelize([ row['pca'] for row in test_result_rdd.collect() ])
test_y_rdd = sc.parallelize([ row['label'] for row in test_result_rdd.collect() ])
train_x_rdd = sc.parallelize([ row['pca'] for row in train_result_rdd.collect() ])
train_y_rdd = sc.parallelize([ row['label'] for row in train_result_rdd.collect() ])

In [4]:
# core algorithms

def calculateDistance(test_rdd, train_rdd):
    #num_test = len(test)
    #num_train = len(train)
    num_test = test_rdd.count()
    num_train = train_rdd.count()
    # transform into array
    test = np.array(test_rdd.collect())
    train = np.array(train_rdd.collect())
    dists = np.zeros((num_test, num_train))
    for i in range(num_test):
        test_i  = test[i, :]
    #print(test_i)
    #dists[i, :] = np.sqrt(np.power(train_ - test_i, 2).sum(axis=1))
        dists[i, :] = np.sqrt(np.sum(np.power(train - test_i, 2), 1))
    return dists
#print(dists)  

def predictLabels(dists, train_y_rdd, k):
    train_y = np.array(train_y_rdd.collect())
    num_test = dists.shape[0]
    y_pred = np.zeros(num_test, dtype=int)
    for i in range(num_test):
        #A list of length k storing the labels of the k nearest neighbors to the ith test point.
        closest_y = []
        dist_i = dists[i, :]
        #print("dist_i")
        #print(dist_i)
        dists_sort = np.argsort(dist_i)
        #print("dists_sort")
        #print(dists_sort)
        indices = dists_sort[:k]
        closest_y = []
        #closest_y = train_y[np.where(dists_sort < k)].tolist()
        for indx in indices:
            y = train_y[indx]
            closest_y+=[y]
            #print(train_y)
        # find most common label
        #print("closest_y")
        #print(closest_y)
        #y_pred[i] = max(set(closest_y), key=closest_y.count)
        from collections import Counter
        y_pred[i] = Counter(closest_y).most_common(1)[0][0]
    #print(y_pred)
    return y_pred

In [ ]:
# Check data size
print(test_x_rdd.count())

In [ ]:
# Run this to measure current data metrics
dists = calculateDistance(test_x_rdd, train_x_rdd)

print(dists)

In [ ]:
y_pred = predictLabels(dists, train_y_rdd, 2)
#print(y_pred)

# Getting metrics
from sklearn import metrics
# Transform RDD
test_y = np.array(test_y_rdd.collect())
print(metrics.accuracy_score(test_y, y_pred))

In [23]:
print("Test...")

Test...

In [5]:
# Run this to evaluate effect of k and d on training data
import time

from sklearn import metrics
k_range = [5, 10]
d_range = [50, 100]
scores = []
supports = []
for d in d_range:
    print("Start setting dimention to d=%d" % d)
    for k in k_range:
        print("Start generating model in PCA for k=%d, d=%d" % (k, d))
        #Start Performance Timer
        start = time.time()
        model = generatePCA(train, d)
        end1 = time.time()
        print("Time: {0:.1f}".format(end1-start))
        print("Start applying model on data...")
        train_result = model.transform(train)
        test_result = model.transform(test)
        end2 = time.time()
        print("Time: {0:.1f}".format(end2-end1))
        
        print("Transforming data to rdd...")
        train_result_rdd = train_result.select(["label", "pca"]).rdd
        test_result_rdd = test_result.select(["label", "pca"]).rdd

        test_x_rdd = sc.parallelize([ row['pca'] for row in test_result_rdd.collect() ])
        test_y_rdd = sc.parallelize([ row['label'] for row in test_result_rdd.collect() ])
        train_x_rdd = sc.parallelize([ row['pca'] for row in train_result_rdd.collect() ])
        train_y_rdd = sc.parallelize([ row['label'] for row in train_result_rdd.collect() ])
        end3 = time.time()
        print("Time: {0:.1f}".format(end3-end2))
        
        print("Start calculating distance for k=%d" % k)
        dists = calculateDistance(test_x_rdd, train_x_rdd)
        end4 = time.time()
        print("Time: {0:.1f}".format(end4-end3))
        print("Start predicting on test data...")
        y_pred = predictLabels(dists, train_y_rdd, k)
        end5 = time.time()
        print("Time: {0:.1f}".format(end4-end3))
        end = time.time()

        # Evaluate metrics
        test_y = np.array(test_y_rdd.collect())
        score = metrics.accuracy_score(test_y, y_pred)
        support = metrics.precision_recall_fscore_support(test_y, y_pred)
        supports.append(support)
        scores.append(score)
        print("Dimensions: {0}, Knns: {1} - Accuracy: {2:.2f}, time: {3:.1f}".format(d, k,  score, end-start))
        

print(scores)


Start setting dimention to d=50
Start generating model in PCA for k=5, d=50
Time: 22.3
Start applying model on data...
Time: 1.1
Transforming data to rdd...
Time: 13.3
Start calculating distance for k=5
Time: 1060.5
Start predicting on test data...
Time: 1060.5
Dimensions: 50, Knns: 5 - Accuracy: 0.96, time: 1150.5
Start generating model in PCA for k=10, d=50
Time: 17.4
Start applying model on data...
Time: 1.0
Transforming data to rdd...
Time: 12.6
Start calculating distance for k=10
Time: 1049.6
Start predicting on test data...
Time: 1049.6
Dimensions: 50, Knns: 10 - Accuracy: 0.96, time: 1137.1
Start setting dimention to d=100
Start generating model in PCA for k=5, d=100
Time: 16.9
Start applying model on data...
Time: 0.9
Transforming data to rdd...
Time: 15.1
Start calculating distance for k=5
Time: 2101.9
Start predicting on test data...
Time: 2101.9
Dimensions: 100, Knns: 5 - Accuracy: 0.96, time: 2194.4
Start generating model in PCA for k=10, d=100
Time: 16.0
Start applying mod

In [25]:
sc.appName

sc.master

print(conf.getAll())

help(sc.statusTracker)


[('spark.master', 'spark://localhost:7077'), ('spark.app.name', 'appName'), ('spark.submit.deployMode', 'client'), ('spark.executor.instances', '8'), ('spark.ui.showConsoleProgress', 'true')]
Help on method statusTracker in module pyspark.context:

statusTracker() method of pyspark.context.SparkContext instance
    Return :class:`StatusTracker` object



In [29]:
print(len(y_pred))
print(supports)

10000
[(array([0.97386935, 0.97914857, 0.96666667, 0.9495549 , 0.96028513,
       0.95676906, 0.95995893, 0.96192385, 0.94536082, 0.92745098]), array([0.98877551, 0.99295154, 0.95542636, 0.95049505, 0.96028513,
       0.94282511, 0.97599165, 0.93385214, 0.94147844, 0.93756194]), array([0.98126582, 0.98600175, 0.96101365, 0.95002474, 0.96028513,
       0.94974591, 0.9679089 , 0.94768016, 0.94341564, 0.93247905]), array([ 980, 1135, 1032, 1010,  982,  892,  958, 1028,  974, 1009])), (array([0.97085427, 0.97743056, 0.96460177, 0.95069034, 0.96197328,
       0.94960806, 0.96197328, 0.95791583, 0.9468196 , 0.91910331]), array([0.98571429, 0.99207048, 0.9505814 , 0.95445545, 0.95315682,
       0.95067265, 0.97703549, 0.92996109, 0.93223819, 0.9345887 ]), array([0.97822785, 0.98469611, 0.95754026, 0.95256917, 0.95754476,
       0.95014006, 0.96944588, 0.94373149, 0.93947232, 0.92678133]), array([ 980, 1135, 1032, 1010,  982,  892,  958, 1028,  974, 1009])), (array([0.97091274, 0.98086957, 0.9

In [15]:
print(metrics.classification_report(test_y, y_pred))

             precision    recall  f1-score   support

          0       0.97      0.99      0.98       980
          1       0.97      0.99      0.98      1135
          2       0.96      0.95      0.96      1032
          3       0.95      0.95      0.95      1010
          4       0.96      0.95      0.96       982
          5       0.95      0.95      0.95       892
          6       0.96      0.98      0.97       958
          7       0.95      0.94      0.94      1028
          8       0.95      0.93      0.94       974
          9       0.93      0.93      0.93      1009

avg / total       0.96      0.96      0.96     10000

